In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
#Reading tables from website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dataframe_list = pd.read_html(url, flavor='bs4')

In [3]:
#Data is on first table
table = dataframe_list[0]

In [4]:
#Converting table to array
data = table.to_numpy()

In [5]:
#Converting to 1D array
data = data.reshape(data.shape[0]*data.shape[1],)

In [6]:
data_df = pd.DataFrame(data)

In [7]:
# all info in 'all' column
data_df.columns = ['all']

In [8]:
# "PostCode" extracted
data_df['PostalCode'] = data_df['all'].str.extract('(.{3})',expand=False)

In [9]:
data_df.head()

,all,PostalCode
0,M1ANot assigned,M1A
1,M2ANot assigned,M2A
2,M3ANorth York(Parkwoods),M3A
3,M4ANorth York(Victoria Village),M4A
4,M5ADowntown Toronto(Regent Park / Harbourfront),M5A


In [10]:
#all info except 'PostalCode' in in temp 'rest' column
data_df['rest']=data_df['all'].str.replace(r'^(.{3})','')

<ipython-input-10-b6930228a476>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['rest']=data_df['all'].str.replace(r'^(.{3})','')


In [11]:
data_df.drop(columns=['all'],inplace=True)

In [12]:
data_df.shape

(180, 2)

In [13]:
# records with no 'Borough' removed
data_df = data_df[data_df['rest'] != 'Not assigned']

In [14]:
data_df.shape

(103, 2)

In [15]:
# "Borough" extracted
data_df['Borough'] = data_df['rest'].str.extract('(.+?)\(.+',expand=False)

In [16]:
data_df.head()

,PostalCode,rest,Borough
2,M3A,North York(Parkwoods),North York
3,M4A,North York(Victoria Village),North York
4,M5A,Downtown Toronto(Regent Park / Harbourfront),Downtown Toronto
5,M6A,North York(Lawrence Manor / Lawrence Heights),North York
6,M7A,Queen's Park(Ontario Provincial Government),Queen's Park


In [17]:
# 'Borough' data removed from 'rest'
data_df['rest'] = data_df['rest'].str.replace('^[^(]+','')

<ipython-input-17-2508a9661765>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['rest'] = data_df['rest'].str.replace('^[^(]+','')


In [18]:
# "Neighborhood" extracted
data_df['Neighborhood'] = data_df['rest'].str.extract('\((.+)\).{0,}',expand=False)

In [19]:
data_df.head()

,PostalCode,rest,Borough,Neighborhood
2,M3A,(Parkwoods),North York,Parkwoods
3,M4A,(Victoria Village),North York,Victoria Village
4,M5A,(Regent Park / Harbourfront),Downtown Toronto,Regent Park / Harbourfront
5,M6A,(Lawrence Manor / Lawrence Heights),North York,Lawrence Manor / Lawrence Heights
6,M7A,(Ontario Provincial Government),Queen's Park,Ontario Provincial Government


In [20]:
#'rest' column dropped
data_df.drop(columns=['rest'],inplace=True)

In [21]:
# index reset
data_df.reset_index(drop=True,inplace=True)

In [22]:
data_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [23]:
data_df.shape

(103, 3)

In [24]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')

In [25]:
coordinates_df.shape

(103, 3)

In [26]:
data_df.sort_values(by=['PostalCode'],inplace=True)

In [27]:
coordinates_df.sort_values(by=['Postal Code'],inplace=True)

In [28]:
data_df.reset_index(inplace=True,drop=True)

In [29]:
coordinates_df.reset_index(inplace=True,drop=True)

In [30]:
data_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...
101,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...


In [31]:
coordinates_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [32]:
data_df['Latitude'] = coordinates_df['Latitude']

In [33]:
data_df['Longitude'] = coordinates_df['Longitude']

In [34]:
data_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [35]:
data_df['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East Toronto                                                     4
East York                                                        4
EtobicokeNorthwest                                               1
Queen's Park                                                     1
Downtown TorontoStn A PO Boxes25 The Esplanade                   1
MississaugaCanada Post Gateway Processing Centre                 1
East YorkEast Toronto                                            1
East TorontoBusiness reply mail Processing Centre969 Eastern  

In [41]:
data_df = data_df[~data_df['Borough'].isin(['EtobicokeNorthwest','Downtown TorontoStn A PO Boxes25 The Esplanade','MississaugaCanada Post Gateway Processing Centre','East YorkEast Toronto','East TorontoBusiness reply mail Processing Centre969 Eastern'])]

In [42]:
data_df.reset_index(inplace=True,drop=True)

In [43]:
data_df['Borough'].value_counts()

North York          24
Downtown Toronto    17
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
York                 5
East Toronto         4
East York            4
Queen's Park         1
Name: Borough, dtype: int64

In [46]:
data_toronto_df = data_df[data_df['Borough'].str.contains('Toronto')]

In [48]:
data_toronto_df.reset_index(inplace=True,drop=True)

In [49]:
data_toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,Moore Park / Summerhill East,43.689574,-79.383160
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049


In [50]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


In [51]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [53]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_toronto_df['Latitude'], data_toronto_df['Longitude'], data_toronto_df['Borough'], data_toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Explore Neighborhoods in Toranto

In [107]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [55]:
def getAuthParams():
    return f'client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}'

In [56]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toranto_venues = getNearbyVenues(list(data_toronto_df['Neighborhood']),list(data_toronto_df['Latitude']),list(data_toronto_df['Longitude']))

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / The Junction So

In [68]:
toranto_venues.shape

(1454, 7)

In [69]:
toranto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [70]:
toranto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,15,15,15,15,15,15
Church and Wellesley,72,72,72,72,72,72
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,9,9,9,9,9,9


In [71]:
print('There are {} uniques categories.'.format(len(toranto_venues['Venue Category'].unique())))

There are 224 uniques categories.


## Analyze Each Neighborhood

In [72]:
toranto_venues['Venue Category'].shape

(1454,)

In [76]:
# one hot encoding
toranto_onehot = pd.get_dummies(toranto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toranto_onehot['Neighborhood'] = toranto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toranto_onehot.columns[-1]] + list(toranto_onehot.columns[:-1])
toranto_onehot = toranto_onehot[fixed_columns]

toranto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
toranto_onehot.shape

(1454, 224)

In [78]:
toranto_grouped = toranto_onehot.groupby('Neighborhood').mean().reset_index()
toranto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.015873,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.015873,0.000000,0.000000,0.015873
4,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.027778,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.013889,0.000000,...,0.013889,0.013889,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Commerce Court / Victoria Hotel,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000
7,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.029412,0.000000,...,0.000000,0.000000,0.000000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,Dufferin / Dovercourt Village,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [79]:
toranto_grouped.shape

(36, 224)

In [80]:
num_top_venues = 5

for hood in toranto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toranto_grouped[toranto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2          Bakery  0.05
3        Pharmacy  0.03
4  Farmers Market  0.03


----Brockton / Parkdale Village / Exhibition Place----
                venue  freq
0                Café  0.14
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.05
4             Stadium  0.05


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
                venue  freq
0      Airport Lounge  0.12
1     Airport Service  0.12
2    Airport Terminal  0.12
3    Sculpture Garden  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.05
2                Café  0.05
3      Sandwich Place  0.05
4         Salad Place  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3     Baby S

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toranto_grouped['Neighborhood']

for ind in np.arange(toranto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toranto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Breakfast Spot
1,Brockton / Parkdale Village / Exhibition Place,Café,Breakfast Spot,Coffee Shop,Italian Restaurant,Stadium,Furniture / Home Store,Climbing Gym,Bar,Restaurant,Burrito Place
2,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport Food Court,Airport Gate,Coffee Shop,Harbor / Marina,Airport,Bar
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Japanese Restaurant,Department Store,Burger Joint,Bubble Tea Shop,Thai Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Candy Store,Italian Restaurant,Restaurant,Nightclub,Coffee Shop,Molecular Gastronomy Restaurant


## Cluster Neighborhoods

In [100]:
# set number of clusters
kclusters = 3

toranto_grouped_clustering = toranto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toranto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toranto_merged = data_toronto_df

# merge toranto_grouped with toranto_data to add latitude/longitude for each neighborhood
toranto_merged = toranto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toranto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Moving Target,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Restaurant,Yoga Studio,Frozen Yogurt Shop,Fruit & Vegetable Store
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,0,Fast Food Restaurant,Sandwich Place,Pub,Brewery,Fish & Chips Shop,Park,Liquor Store,Steakhouse,Restaurant,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Brewery,Gastropub,Café,American Restaurant,Bakery,Cheese Shop,Clothing Store,Comfort Food Restaurant,Convenience Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Business Service,Swim School,Yoga Studio,Movie Theater,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


In [102]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toranto_merged['Latitude'], toranto_merged['Longitude'], toranto_merged['Neighborhood'], toranto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [103]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 0, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Yoga Studio,Moving Target,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Bookstore,Restaurant,Yoga Studio,Frozen Yogurt Shop,Fruit & Vegetable Store
2,East Toronto,0,Fast Food Restaurant,Sandwich Place,Pub,Brewery,Fish & Chips Shop,Park,Liquor Store,Steakhouse,Restaurant,Movie Theater
3,East Toronto,0,Coffee Shop,Brewery,Gastropub,Café,American Restaurant,Bakery,Cheese Shop,Clothing Store,Comfort Food Restaurant,Convenience Store
4,Central Toronto,0,Park,Bus Line,Business Service,Swim School,Yoga Studio,Movie Theater,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
5,Central Toronto,0,Hotel,Gym / Fitness Center,Park,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Pizza Place,Hotel Bar,Music Venue
6,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Café,Spa,Salon / Barbershop,Restaurant,Pet Store,Park
7,Central Toronto,0,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Coffee Shop,Gym,Café,Sushi Restaurant,Brewery,Pharmacy
9,Central Toronto,0,Coffee Shop,Liquor Store,Light Rail Station,Restaurant,Bank,Bagel Shop,Pub,Supermarket,Fried Chicken Joint,American Restaurant
11,Downtown Toronto,0,Coffee Shop,Pizza Place,Café,Italian Restaurant,Restaurant,Pub,Park,Bakery,Yoga Studio,Deli / Bodega


#### Cluster 2

In [104]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 1, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Restaurant,Playground,Park,Trail,Moving Target,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
10,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Movie Theater,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
23,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Moroccan Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant


#### Cluster 3

In [105]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 2, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Home Service,Fast Food Restaurant,Moving Target,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
